In [1]:
%matplotlib inline

In [2]:
from halotools.sim_manager import CachedHaloCatalog
halocat = CachedHaloCatalog(simname='bolplanck')

from halotools.empirical_models import PrebuiltSubhaloModelFactory
model = PrebuiltSubhaloModelFactory('behroozi10')

model.populate_mock(halocat)

In [3]:
mask = model.mock.galaxy_table['stellar_mass'] > 10**9.5
source_galaxies = model.mock.galaxy_table[mask]
host_halo_mask = source_galaxies['halo_upid'] == -1
source_halos = source_galaxies[host_halo_mask]

target_halos = halocat.halo_table[halocat.halo_table['halo_upid'] == -1]

In [4]:
from galsampler import halo_bin_indices

mpeak_bins = np.logspace(11, 15, 25)

source_halos['bin_number'] = halo_bin_indices(mass=(source_halos['halo_mpeak'], mpeak_bins))
target_halos['bin_number'] = halo_bin_indices(mass=(target_halos['halo_mpeak'], mpeak_bins))

In [5]:
from galsampler.host_halo_selection import compute_richness

source_halos['richness'] = compute_richness(source_halos['halo_id'], 
                                            source_galaxies['halo_hostid'])

In [6]:
from galsampler import source_halo_index_selection

nhalo_min = 50
source_halo_selection_indices = source_halo_index_selection(source_halos['bin_number'],
                                                target_halos['bin_number'],
                                                nhalo_min, mpeak_bins)
# selected_halos = source_halos[selection_indices]

In [7]:
num_target_gals = np.sum(source_halos['richness'][source_halo_selection_indices])

In [8]:
from halotools.utils import crossmatch

idxA, idxB = crossmatch(source_galaxies['halo_hostid'].data, source_halos['halo_id'].data)

source_galaxies['bin_number'] = 0
source_galaxies['richness'] = 0

source_galaxies['bin_number'][idxA] = source_halos['bin_number'][idxB]
source_galaxies['richness'][idxA] = source_halos['richness'][idxB]

source_galaxies.sort(['bin_number', 'halo_hostid'])

In [9]:
selected_halo_ids = source_halos['halo_id'][source_halo_selection_indices]

source_galaxy_selection_indices, __ = crossmatch(source_galaxies['halo_hostid'].data, 
                        source_halos['halo_hostid'].data)

